In [51]:
# 匯入套件
import numpy as np
import pandas as pd
import itertools
import gc

In [41]:
# 定義function

# 此function為網路上的人撰寫，目的是降低DataFrame的大小
def reduce_mem_usage(df, silent=True, allow_categorical=True, float_dtype="float32"):
    """
    Iterates through all the columns of a dataframe and downcasts the data type
     to reduce memory usage. Can also factorize categorical columns to integer dtype.
    """
    def _downcast_numeric(series, allow_categorical=allow_categorical):
        """
        Downcast a numeric series into either the smallest possible int dtype or a specified float dtype.
        """
        if pd.api.types.is_sparse(series.dtype) is True:
            return series
        elif pd.api.types.is_numeric_dtype(series.dtype) is False:
            if pd.api.types.is_datetime64_any_dtype(series.dtype):
                return series
            else:
                if allow_categorical:
                    return series
                else:
                    codes, uniques = series.factorize()
                    series = pd.Series(data=codes, index=series.index)
                    series = _downcast_numeric(series)
                    return series
        else:
            series = pd.to_numeric(series, downcast="integer")
        if pd.api.types.is_float_dtype(series.dtype):
            series = series.astype(float_dtype)
        return series

    if silent is False:
        start_mem = np.sum(df.memory_usage()) / 1024 ** 2
        print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    if df.ndim == 1:
        df = _downcast_numeric(df)
    else:
        for col in df.columns:
            df.loc[:, col] = _downcast_numeric(df.loc[:,col])
    if silent is False:
        end_mem = np.sum(df.memory_usage()) / 1024 ** 2
        print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
        print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

# 讀取資料，by chunk
def read_data_by_chunk(path, df_test, col_list, shop_tag_list, chunksize):
    reader = pd.read_csv(path,
                        error_bad_lines=False, # 會自動忽略錯誤row
                        # header=None, # 看資料有無欄位名稱
                        iterator=True,
                        usecols=col_list  #限縮要取的資料欄位
                        )
    loop = True
    chunks = []
    while loop:
        try:
            chunk = reader.get_chunk(chunksize)
            chunk = chunk[chunk.chid.isin(df_test.chid)]    #只取測試資料的前10,000 chid
            chunk = chunk[chunk.shop_tag.isin(shop_tag_list)]   #指取要預測的消費類別
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    df = pd.concat(chunks,ignore_index=True)
    return df


In [42]:
# 預先定義好的list，包含要預測的shop tag或目前要使用的columns
SHOP_TAG_LIST = ['2', '6', '10', '12',
                '13', '15', '18', '19',
                '21', '22', '25', '26',
                '36', '37', '39', '48']
COL_LIST = ['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'card_1_txn_cnt',
       'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt',
       'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt',
       'card_10_txn_cnt', 'card_11_txn_cnt', 'card_12_txn_cnt',
       'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt',
       'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
       'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
       'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
       'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
       'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct']

#### 讀取測試資料，並篩選前10,000個chid

In [43]:
df_test = pd.read_csv(r'D:\Gallon\Project\07_E_SUN_2021_Winter_AI\data\需預測的顧客名單及提交檔案範例.csv')
df_test = df_test[:10000].reset_index(drop=True)
df_test

,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26
...,...,...,...,...
9995,10203858,25,22,37
9996,10008113,22,48,15
9997,10053154,37,39,12
9998,10252782,26,18,12


#### 匯入訓練資料，並根據篩選後的測試資料(df_test)選擇部分chid

In [44]:
df = read_data_by_chunk(path=r'D:\Gallon\Project\07_E_SUN_2021_Winter_AI\data\tbrain_cc_training_48tags_hash_final.csv',
                    df_test=df_test,
                    col_list=COL_LIST,
                    shop_tag_list=SHOP_TAG_LIST,
                    chunksize=100000)

Iteration is stopped.


In [45]:
# 壓縮資料大小
df = reduce_mem_usage(df, silent=False)

Memory usage of dataframe is 144.00 MB
Memory usage after optimization is: 49.81 MB
Decreased by 65.4%


#### 建立以(df, chid, shop_tag)為index 的資料

In [46]:
indexlist = []
x = itertools.product(df.dt.unique(), df_test.chid.unique(), shop_tag_list)
indexlist.append(np.array(list(x)))
matrix = pd.DataFrame(
        data=np.concatenate(indexlist, axis=0),
        columns=["dt", "chid", "shop_tag"])

In [47]:
# 改變資料型態，壓縮資料大小
matrix.dt = matrix.dt.astype('int8')
matrix.chid = matrix.chid.astype('int32')
matrix.shop_tag = matrix.shop_tag.astype('int8')
df.shop_tag = df.shop_tag.astype('int8')

In [48]:
# 將訓練資料合併進我們建立的以(df, chid, shop_tag)為index 的資料
matrix = matrix.merge(df, on=['dt', 'chid', 'shop_tag'], how='left')

In [49]:
# 資料補0，並壓縮資料
matrix.fillna(0, inplace=True)
matrix = reduce_mem_usage(matrix, silent=False)

Memory usage of dataframe is 930.18 MB
Memory usage after optimization is: 439.45 MB
Decreased by 52.8%


In [50]:
del df

In [52]:
gc.collect()

215

In [53]:
matrix.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3840000 entries, 0 to 3839999
Data columns (total 43 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   dt                        int8   
 1   chid                      int32  
 2   shop_tag                  int8   
 3   txn_cnt                   int16  
 4   txn_amt                   float32
 5   domestic_offline_cnt      int16  
 6   domestic_online_cnt       int8   
 7   overseas_offline_cnt      int8   
 8   overseas_online_cnt       int16  
 9   domestic_offline_amt_pct  float32
 10  domestic_online_amt_pct   float32
 11  overseas_offline_amt_pct  float32
 12  overseas_online_amt_pct   float32
 13  card_1_txn_cnt            int8   
 14  card_2_txn_cnt            int8   
 15  card_3_txn_cnt            int8   
 16  card_4_txn_cnt            int16  
 17  card_5_txn_cnt            int8   
 18  card_6_txn_cnt            int16  
 19  card_7_txn_cnt            int8   
 20  card_8_txn_cnt          

In [56]:
matrix.head()

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct
0,1,10128239,2,0,0.0,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,10128239,6,0,0.0,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,10128239,10,0,0.0,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,10128239,12,0,0.0,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,10128239,13,0,0.0,0,0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
